### Imports

In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from umap import UMAP
import numpy as np
import warnings
warnings.filterwarnings("ignore") # Some log2(0) in the DE analysis raises runtime errors, but we supress because it just means its not significant

/home/cm4ai/miniconda3/envs/cm4ai/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Reading in Data

In [12]:
%%time
#Change this to the appropriate data path
data_path = '/home/cm4ai/challenges/perturbseq/data/KOLF_Chroma_mixscape_output_x_pert.h5ad'
# data_path = '/home/cm4ai/challenges/perturbseq/data/cm4ai_chromatin_kolf2.1j_undifferentiated_untreated_crispr_4channel_0.1_alpha/KOLF2.1J_undifferentiated_untreated_chromatin.h5ad'
adata = ad.read_h5ad(data_path) #Note that the .X is already set to the X_pert layer if you are following the single cell best practices guide

CPU times: user 1.13 s, sys: 1.36 s, total: 2.49 s
Wall time: 42.4 s


In [14]:
adata.obs.mixscape_class_global


AAACCCAAGTACAGAT-1    NP
AAACCCAGTTAGCGGA-1    KO
AAACCCAGTTAGGGTG-1    KO
AAACCCAGTTCGGCGT-1    NP
AAACCCATCAGAGTTC-1    NP
                      ..
TTTGTTGCACACTTAG-4    NP
TTTGTTGCACGCGCAT-4    NP
TTTGTTGGTGATCGTT-4    KO
TTTGTTGGTGCAACAG-4    KO
TTTGTTGTCTGGCCGA-4    KO
Name: mixscape_class_global, Length: 34326, dtype: category
Categories (3, object): ['KO', 'NP', 'NT']

In [31]:
adata_ko = adata[adata.obs['mixscape_class_global'].isin(["KO"])]
# adata_ko.layers["X_pert"][0].data
adata_ko

View of AnnData object with n_obs × n_vars = 16899 × 6000
    obs: 'celltype', 'perturbation_type', 'n_counts', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'ribo_outlier', 'channel', 'predicted_doublets', 'doublet_score', 'n_gRNA', 'guide_labels', 'perturbation_target_genes', 'guide_ID', 'gene_target', 'perturbation', 'S_score', 'G2M_score', 'phase', 'mixscape_class_p_ko', 'mixscape_class', 'mixscape_class_global'
    var: 'gene_ids', 'feature_types', 'n_counts', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
  

### Visualizing perturbation similarities through correlation analysis
1. The data provided to you has been filtered through the entire [single cell best practices](https://www.sc-best-practices.org/conditions/perturbation_modeling.html#analysing-single-pooled-crispr-screens) pipeline as described in the earlier talk (link).
2. Download the data (link) and to read the .h5ad into Python. Read up on the data structure of [AnnData objects](https://anndata.readthedocs.io/en/latest/) and explore the .obs/.var of the downloaded data. The .X is the X_pert layer computed by [pertpy](https://pertpy.readthedocs.io/en/latest/tutorials/notebooks/mixscape.html), and is filtered down to top 6000 highly variable genes.
3. Compute a mean gene expression vector for each perturbation.
4. Compute the pairwise Pearson correlation matrix between all perturbations.
5. Read on generally what [UMAP](https://umap-learn.readthedocs.io/en/latest/) does. Use the correlation matrix as a precomputed feature matrix as input to UMAP to get a 2-dimensional embedding of each perturbation. 6. Use the nearest n=3 neighbors with spread = 1.0 to compute the UMAP.
7. Use [networkx](https://networkx.org/documentation/stable/tutorial.html) to plot each perturbation as a node, using the UMAP embeddings as the X/Y position of each node. Draw edges between each node and the top 5 person correlates.
8. What perturbations cluster together in the network? Do these have known biological relationships? How does the clustering change as a function of changing the number of UMAP neighbors?

In [50]:
target_genes_category='perturbation_target_genes'

target_genes = adata_ko.obs[target_genes_category].unique()
pert = adata_ko[adata_ko.obs[target_genes_category] == target_genes[0], :]

# print(pert.layers["X_pert"])
# print(adata_ko.obs[target_genes_category])
# print(pert.obs)
print(pert.to_df())
# print(pert.to_df().mean())

                      CFAP74  AL590822.1    CCDC27  AL139823.1      ESPN  \
AAACCCAGTTAGCGGA-1  0.000000   -0.036336  0.000000         0.0 -0.130531   
AACCTGACAGGCACAA-1  0.000000    0.000000  0.000000         0.0  0.279643   
ACCCAAAGTTCCACGG-1 -0.022707    0.000000  0.000000         0.0  0.387503   
AGAAGCGGTGTTGCCG-1 -0.062174    0.000000 -0.028884         0.0  0.754528   
AGACCATAGTACTGTC-1  0.000000   -0.029467 -0.035423         0.0 -0.771195   
...                      ...         ...       ...         ...       ...   
TGAACGTAGGGTTGCA-4 -0.036524    0.000000 -0.025254         0.0 -0.052724   
TGACAGTTCGAAGGAC-4 -0.153828    0.000000 -0.052731         0.0 -0.130408   
TGTGCGGCAACAAGAT-4 -0.035887    0.000000 -0.045455         0.0 -0.180586   
TTACCGCGTATCGATC-4 -0.036524    0.000000  0.000000         0.0  0.298990   
TTCACGCTCGCTTAAG-4 -0.044501    0.000000 -0.039629         0.0 -0.607604   

                      KLHL21  LINC01672      PER3  TNFRSF9    ERRFI1  ...  \
AAACCCAGTT

In [40]:
%%time
def create_perturbation_network(adata, target_genes_category='perturbation_target_genes'):
    # Filter cells with mixscape_class_global as 'KO' which correspond to perturbed cells
    # Note that "KO" actually represents a Knock-Down in this application as CRISPRi is used
    adata_ko = adata[adata.obs['mixscape_class_global'].isin(["KO"])].copy()
    
    # Get the list of perturbations
    target_genes = adata_ko.obs[target_genes_category].unique()
    print("Target genes")
    print(target_genes)
    
    # Compute average gene expression vector for all cells with each perturbation
    avg_gene_expr = {}
    for gene in target_genes:
        pert = adata_ko[adata_ko.obs[target_genes_category].isin([gene]), :]
        # print(pert)
        avg_gene_expr[gene] = pert.to_df().mean()

    print("Average expression")
    print(avg_gene_expr)

    # Create DataFrame for the expression data
    expression_df = pd.DataFrame(avg_gene_expr)
    # print(expression_df)

    # Compute pairwise Pearson correlation matrix between all perturbation average gene expression vectors
    
    cor_mat = expression_df.corr(method="pearson")
    print("Pearson correlation")
    print(cor_mat)

    # Convert correlation to distance; want a small distance between high correlates
    dist_mat = 1 - cor_mat
    print("Distance")
    print(dist_mat)

    # Compute the 2D UMAP embedding for each perturbation, using the correlation vector as the feature vector for each perturbation
    umap = UMAP(n_components=2, metric="precomputed", n_neighbors=3, min_dist=0.1, spread=10)
    positions = umap.fit_transform(dist_mat)
    print("UMAP position")
    print(positions)

    # Store 2D embeddings as positions in a dictionary for NetworkX Visualization
    pos_dict = {gene: positions[i] for i, gene in enumerate(cor_mat.columns)}

    # Initialize the networkx graph, where nodes are each perturbation
    G = nx.Graph()

    # Add nodes and edges, positioning nodes based on the UMAP, and draw edges between each perturbation and the top 5 pearson correlates
    for gene in cor_mat.columns:
        G.add_node(gene)
        nn_idx = np.argsort(-np.abs(cor_mat.loc[gene].values))[:6]

    # Visualize the graph
    
    print("Done")

create_perturbation_network(adata)

Target genes
['PRKAG2', 'HDAC10', 'KDM6A', 'MSL1', 'HDAC7', ..., 'HDAC2', 'PRMT2', 'BRE1A', 'KDM6B', 'BRCA1']
Length: 106
Categories (106, object): ['ARDT1', 'ARTD1', 'ATM', 'ATR', ..., 'YWHAG', 'YWHAH', 'YWHAQ', 'YWHAZ']
Average expression
{'PRKAG2': CFAP74       -0.022573
AL590822.1   -0.006434
CCDC27       -0.008953
AL139823.1    0.000000
ESPN         -0.030212
                ...   
MT-ND3       -0.102203
MT-ND4       -0.062105
MT-ND5       -0.131091
MT-ND6       -0.125680
MT-CYB       -0.095149
Length: 6000, dtype: float32, 'HDAC10': CFAP74       -0.018084
AL590822.1   -0.009030
CCDC27       -0.008495
AL139823.1    0.000000
ESPN         -0.076727
                ...   
MT-ND3       -0.107084
MT-ND4       -0.089526
MT-ND5       -0.303000
MT-ND6       -0.201648
MT-CYB       -0.122694
Length: 6000, dtype: float32, 'KDM6A': CFAP74       -0.002065
AL590822.1   -0.007482
CCDC27       -0.011273
AL139823.1    0.000000
ESPN         -0.055334
                ...   
MT-ND3       -0.095289
MT

### Computing Differentially Expressed Genes

1. For each perturbation, compute the number of differentialy expressed genes relative to NTCs. What is an appropriate DE test to use (e.g. parametric or non-parametric?) What are the FDR and Log Folders Change cutoffs chosen? Justify the selection
2. Investigate the DEGs for a perturbation of interest. Is there any interesting biological relationship between them?
3. Can you leverage parallel computing to significantly speed up the differential expression analysis?

In [ ]:
%%time
def create_differential_gene_expression_plot(adata, pvalue_threshold=-np.log10(0.05)):

    # Isolate only cells identified as having a mixscape_class_global of "KO" or "NT" corresponding to perturbed or non targeting controls
    # Note that "KO" actually represents a Knock-Down in this application as CRISPRi is used
    adata_ko = adata[adata.obs['mixscape_class_global'].isin(['NT',"KO"])].copy()

    # Extract a list of each perturbed gene
    genes_of_interest = [] # Adjust this accordingly

    # Initialize datastructure to store the number of significiantly upregulated / downregulated genes
    

    for gene_of_interest in genes_of_interest: # TODO: Speed this up (e.g. parallel processing)
        # Isolate cells that are either NTCs or having a perturbation corresponding to the gene of interest
        

        # Step 2: Differential Expression Analysis using sc.tl.rank_genes_groups (what is an appropriate DE test to use? why?)
        

        # Step 3: Extract the results from the DE analysis
        

       # Step 4: Count the number of significantly upregulated and downregulated genes
        

    # Plot the Number of significantly upregulated and downregulated genes per perturbation
   
    print("Done")

create_differential_gene_expression_plot(adata)